In [2]:
import pandas as pd
from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
import re
import json
from pymed import PubMed

In [12]:
email = 'fcc-pesquisas@hotmail.com'
query = '(Sars-cov2 OR COVID OR nCOV)'
#query = 'Monkeypox'
pubmed = PubMed(tool="PyMed", email="fcc-pesquisas@hotmail.com")
n = pubmed.getTotalResultsCount(query=query)
results = pubmed.query(query, max_results=99999999999)
print(n)

283370


In [9]:
for r in results:
    print(r.abstract)
    break

Obesity is considered an important comorbidity for a range of noninfectious and infectious disease states including those that originate in the lung, yet the mechanisms that contribute to this susceptibility are not well defined. In this study, we used the diet-induced obesity (DIO) mouse model and two models of acute pulmonary infection, 


In [2]:
results = pd.read_csv('Dados/saida.csv', sep='\t')
results.sample(5)

,pubmed_id,titulo,titulo_checagem,issn,journal,doi,abstract
320,33127507,Comparison of antiviral effect for mild-to-mod...,Comparison of antiviral effect for mild-to-mod...,1878-3511,International journal of infectious diseases :...,10.1016/j.ijid.2020.10.062,The viral shedding duration was similar betwee...
541,33279832,Pros and cons for use of statins in people wit...,Letter in response to the article: Pros and co...,1878-0334,Diabetes & metabolic syndrome,10.1016/j.dsx.2020.11.017,"COVID-19, transmitted by SARS-CoV-2, is one of..."
874,32321905,Neuropsychiatric adverse events of chloroquine...,Neuropsychiatric adverse events of chloroquine...,1881-7823,Bioscience trends,10.5582/bst.2020.03082,"In late March and early April 2020, the antima..."
376,32984847,Long-term hydroxychloroquine use in patients w...,Long-term hydroxychloroquine use in patients w...,2665-9913,The Lancet. Rheumatology,10.1016/S2665-9913(20)30305-2,None.
503,32735768,Anti-Inflammatory Properties of Drugs Used to ...,Anti-Inflammatory Properties of Drugs Used to ...,1482-1826,Journal of pharmacy & pharmaceutical sciences ...,10.18433/jpps31346,COVID-19 infection is associated with systemic...


In [ ]:
n = 15000
translator = Translator(service_urls=['translate.google.com'])
analyzer = SentimentIntensityAnalyzer()
base = dict.fromkeys(['titulo', 'resumo','resumo_traduzido', 'revista', 'data', 'qualis', 'H-index','doi'],[np.nan]*n) | dict.fromkeys(['pro', 'contra', 'indefinido'],[np.nan]*n) 
base = pd.DataFrame(base)
k = 0
for r in results:
    base.at[k,'titulo'] = r.title
    try:
        base.at[k,'resumo'] = r.abstract
    except:
        continue
    try:
        base.at[k,'doi'] = r.doi
    except:
        pass
    try:
        translation = translator.translate(r.abstract, dest='pt')
        base.at[k,'resumo_traduzido'] = translation.text
    except:
        pass
    try:
        base.at[k,'resultados'] = r.results
    except:
        pass
    try:
        base.at[k,'revista'] = r.journal
    except:
        pass
    try:
        base.at[k, 'data'] = r.publication_date
    except:
        pass
    try:
        base.at[k, 'H-index'] = sjr.loc[sjr['Title']==r.journal.lower(),'H index'].values[0]
    except:
        pass
    try:
        base.at[k, 'qualis'] = qualis.loc[qualis['TITULO']==r.journal.lower(),'ESTRATO'].values[0]
    except:
        pass
    if isinstance(r.abstract, str):
        nota = {}
        usados = []
        if isinstance(r.title, str):
            texto = to_lower(' '.join([r.title,r.abstract]))
        else:
            texto = to_lower(r.abstract)
        frases = re.split('\; |\.|\n',texto)
        for i in range(len(frases)):
            palavras = frases[i].replace('\n', ' ').replace('.', '').replace(',','').split()
            for j in range(len(palavras)):
                t = palavras[j]
                if (t in farm):
                    if t not in usados:
                        usados.append(t)
                        nota[t] = 0
                    nota[t] += analyzer.polarity_scores(frases[i])['compound']
        for t in usados:
            if nota[t] > 0.25:
                if isinstance(base.at[k,'pro'], list):
                    base.at[k,'pro'].append(t)
                else:
                    base.at[k,'pro'] = [t]
            elif nota[t] < -0.25:
                if isinstance(base.at[k,'contra'], list):
                    base.at[k,'contra'].append(t)
                else:
                    base.at[k,'contra'] = [t]
            else:
                if isinstance(base.at[k,'indefinido'], list):
                    base.at[k,'indefinido'].append(t)
                else:
                    base.at[k,'indefinido'] = [t]
    k +=1
    if k > n:
        break
    text = np.round(100*k/n,2)
    print(f'{"|"*int(text) + "-"*(100-int(text))}  {text}%', end='\r')
print(f'{"|"*100} 100.00%')
base.to_csv('Pymed_CVD.csv')
valid = base.loc[(~base['pro'].isnull()) | (~base['contra'].isnull()) | (~base['contra'].isnull())]
valid.to_csv('Pymed_validos.csv')

In [4]:
# Passo 3 - Carregar base de dados de fármacos, qualis e H-Index
with open('Dados/drug-drugsfda-0001-of-0001.json') as file:
    fda = json.load(file)
to_lower = lambda x: x.lower()
farmacos = pd.read_csv('Dados/DrugCentral/drug.target.interaction.tsv.gz',sep='\t')
farm = [f.lower() for f in pd.unique(farmacos['DRUG_NAME'])]
druglist = []
for entry in fda['results']:
    if 'products' in entry.keys():
        drugs = entry['products']
        for d in drugs:
            names = [d['brand_name']]
            for ig in d['active_ingredients']:
                names.append(ig['name'])
    druglist.extend(names)
druglist = [d.lower() for d in pd.unique(druglist)]
farm.extend(['vaccine','convalescent plasma'])
farm.extend(druglist)
farm.remove('0')
farm = pd.unique(farm)
qualis = pd.read_csv('Dados/qualis2019.csv', delimiter=';')
qualis['TITULO'] = qualis['TITULO'].apply(to_lower)
sjr = pd.read_csv('Dados/scimago.csv',delimiter=';')
sjr['Title'] = sjr['Title'].apply(to_lower)

In [6]:
base

,titulo,resumo,resumo_traduzido,revista,data,qualis,H-index,doi,pro,contra,indefinido,resultados
0,Monkeypox - Past as Prologue.,NaN,NaN,The New England journal of medicine,2022-08-25,NaN,NaN,10.1056/NEJMe2210535,NaN,NaN,NaN,NaN
1,Spillover zoonotic 'Langya virus': Is it a mat...,NaN,NaN,The veterinary quarterly,2022-08-25,A2,NaN,10.1080/01652176.2022.2117874,NaN,NaN,NaN,NaN
2,Monkeypox.,NaN,NaN,American journal of clinical pathology,2022-08-25,A2,132.0,10.1093/ajcp/aqac100,NaN,NaN,NaN,NaN
3,Monkeypox infection: An update for the practic...,"As of August 5, 2022, >26,000 cases of monkeyp...","Em 5 de agosto de 2022,> 26.000 casos de Monke...",European journal of internal medicine,2022-08-24,NaN,79.0,10.1016/j.ejim.2022.08.022,NaN,NaN,NaN,NaN
4,US CDC announces major changes after criticism...,None,NaN,BMJ (Clinical research ed.),2022-08-24,NaN,NaN,10.1136/bmj.o2074,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1277,Smallpox and monkeypox in non-human primates.,In considering global eradication of smallpox ...,"Ao considerar a erradicação global da varíola,...",Bulletin of the World Health Organization,1968-01-01,NaN,177.0,None,NaN,NaN,NaN,None
1278,Histopathological and virological studies on m...,None,NaN,Archiv fur die gesamte Virusforschung,1967-01-01,NaN,NaN,10.1007/BF01241445,NaN,NaN,NaN,None
1279,PLAQUE MORPHOLOGY OF MONKEYPOX VIRUS AS AN AID...,None,NaN,Journal of bacteriology,1964-01-01,A3,255.0,10.1128/jb.87.1.238-239.1964,NaN,NaN,NaN,None
1280,PROTECTION OF RHESUS MONKEYS AGAINST MONKEYPOX...,None,NaN,American journal of veterinary research,1964-01-01,A3,94.0,None,NaN,NaN,NaN,None


In [29]:
n = len(results)
#base = dict.fromkeys(['titulo', 'resumo','resumo_traduzido','resultados', 'revista', 'data', 'qualis', 'H-index','doi'],[np.nan]*n) | dict.fromkeys(farm,[np.nan]*n) 
base = dict.fromkeys(['titulo', 'resumo','resumo_traduzido', 'revista', 'data', 'qualis', 'H-index','doi'],[np.nan]*n) | dict.fromkeys(['pro', 'contra'],[np.nan]*n) 
base = pd.DataFrame(base)  
translator = Translator(service_urls=['translate.google.com'])
analyzer = SentimentIntensityAnalyzer()
for k in range(n):
    base.at[k,'titulo'] = results.titulo[k]
    try:
        base.at[k,'resumo'] = results.abstract[k]
    except:
        pass
    try:
        base.at[k,'doi'] = results.doi[k]
    except:
        pass
    try:
        translation = translator.translate(results.abstract[k], dest='pt')
        base.at[k,'resumo_traduzido'] = translation.text
    except:
        pass
    try:
        base.at[k,'resultados'] = results.results
    except:
        pass
    try:
        base.at[k,'revista'] = results.journal[k]
    except:
        pass
    try:
        base.at[k, 'data'] = results.publication_date[k]
    except:
        pass
    try:
        base.at[k, 'H-index'] = sjr.loc[sjr['Title']==results.journal[k].lower(),'H index'].values[0]
    except:
        pass
    try:
        base.at[k, 'qualis'] = qualis.loc[qualis['TITULO']==results.journal[k].lower(),'ESTRATO'].values[0]
    except:
        pass
    if isinstance(results.abstract[k], str):
        nota = {}
        usados = []
        if isinstance(results.titulo[k], str):
            texto = to_lower(' '.join([results.titulo[k],results.abstract[k]]))
        else:
            texto = to_lower(results.abstract[k])
        frases = re.split('\; |\.|\n',texto)
        for i in range(len(frases)):
            palavras = frases[i].replace('\n', ' ').replace('.', '').replace(',','').split()
            for j in range(len(palavras)):
                t = palavras[j]
                if (t in farm):
                    if t not in usados:
                        usados.append(t)
                        nota[t] = 0
                    nota[t] += analyzer.polarity_scores(frases[i])['compound']
        for t in usados:
            if nota[t] > 0:
                if isinstance(base.at[k,'pro'], list):
                    base.at[k,'pro'].append(t)
                else:
                    base.at[k,'pro'] = [t]
            else:
                if isinstance(base.at[k,'contra'], list):
                    base.at[k,'contra'].append(t)
                else:
                    base.at[k,'contra'] = [t]
    text = np.round(100*k/n,2)
    print(f'{"|"*int(text) + "-"*(100-int(text))}  {text}%', end='\r')
print(f'{"|"*100} 100.00%')
base.to_csv('teste_CVD.csv')
valid = base.loc[(~base['pro'].isnull()) | (~base['contra'].isnull())]
valid.to_csv('validos.csv')

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 100.00%


In [32]:
valid

,titulo,resumo,resumo_traduzido,revista,data,qualis,H-index,doi,pro,contra
2,Lack of Association of Baseline 25-Hydroxyvita...,Vitamin D deficiency (VDD) owing to its immuno...,Acredita-se que a deficiência de vitamina D (V...,Scientific reports,NaN,A2,242.0,10.1038/s41598-021-85809-y,NaN,"[cholecalciferol, oxygen]"
3,Credible learning of hydroxychloroquine and de...,The assumptions required for a beneficial effe...,As suposições necessárias para um efeito benéf...,medRxiv : the preprint server for health sciences,NaN,NaN,NaN,10.1101/2020.12.06.20244798,[dexamethasone],[hydroxychloroquine]
4,A Phase 1: Safety and Immunogenicity Trial of ...,Safe and effective vaccines against SARS-CoV-2...,Vacinas seguras e eficazes contra SARS-CoV-2 p...,Frontiers in immunology,NaN,NaN,155.0,10.3389/fimmu.2022.898151,[vaccine],NaN
9,The efficacy and safety of hydroxychloroquine ...,Our results showed no significant beneficial e...,Nossos resultados não mostraram efeito benéfic...,Infectious diseases and therapy,NaN,A4,33.0,10.1007/s40121-021-00397-8,NaN,[hydroxychloroquine]
10,Role of ivermectin in the prevention of COVID-...,Two-dose ivermectin prophylaxis at a dose of 3...,Profilaxia de ivermectina em duas doses na dos...,PloS one,NaN,A1,367.0,10.1371/journal.pone.0247163,NaN,[ivermectin]
...,...,...,...,...,...,...,...,...,...,...
907,Continuous Intravenous Anakinra Infusion to Ca...,Continuous infusion of IV anakinra may result ...,A infusão contínua de anakinra IV pode resulta...,ACR open rheumatology,NaN,NaN,NaN,10.1002/acr2.11135,[anakinra],NaN
908,Current knowledge about the antivirals remdesi...,Recent international epidemics of coronavirus-...,NaN,"One health (Amsterdam, Netherlands)",NaN,NaN,NaN,10.1016/j.onehlt.2020.100128,"[remdesivir, vaccine]",NaN
911,Associations between immune-suppressive and st...,The COVID-19 pandemic has led to challenging d...,A pandemia COVID-19 levou a desafiar a tomada ...,Ecancermedicalscience,NaN,NaN,37.0,10.3332/ecancer.2020.1022,"[prednisolone, tacrolimus]",[methotrexate]
913,The potential of low molecular weight heparin ...,"On March 11, 2020, the World Health Organizati...","Em 11 de março de 2020, a Organização Mundial ...",Clinical and translational science,NaN,NaN,48.0,10.1111/cts.12880,NaN,[heparin]


In [12]:
# PASSO 5 - Processamento dos resultados para geração do dashboard

# Gráfico 1 - Papers por data de publicação
df['data'] = pd.to_datetime(df['data'])
#base['ano'] = base['data'].dt.year
#base['mes'] = base['data'].dt.month
df['ano'] = 1800
df['mes'] = 1
datas = df['ano'].value_counts().reset_index().sort_values('index')

# Gráfico 2 - Papers por Qualis
qualis = df['qualis'].value_counts().reset_index().sort_values('index')
qualis.iloc[1:]

# Gráfico 4 - Papers por h-index
hindex = df['H-index'].value_counts().reset_index().sort_values('index')
hist = np.histogram(hindex.iloc[1:]['H-index'],bins=25)
labels_h = hist[1][1:]
data_h = hist[0]

# Gráfico 5, 6 e 8 - ranking dos medicamentos por positivo e negativo
dados = {'medicamento':[], 'positivo':[], 'negativo':[], 'contagem':[], 'comqualis':[]}
for c in df.columns[10:-3]:
    dados['medicamento'].append(c)
    dados['positivo'].append((df[c] == 1).sum())
    dados['negativo'].append((df[c] == 0).sum())
    dados['contagem'].append(dados['negativo'][-1] + dados['positivo'][-1])
    dados['comqualis'].append((df.loc[df['qualis'].isin(['A1','A2','A3','A4','B1','B2','B3','B4']),c]>=0).sum())
dados = pd.DataFrame(dados)
dados = dados.sort_values(['contagem','negativo','positivo'], ascending=False)
medicamentos = dados.iloc[:25]

# PASSO 6 - Preparação do arquivo dashboard.js
file = open('dashboard.js')
lines = file.readlines()
l1 = lines[:13]
d1 = lines[26:29]
l2 = lines[42:70]
d2 = lines[75:79]
l3 = lines[84:114]
d3 = lines[123:127]
l4 = lines[136:166]
d4 = lines[186:189]
l5 = lines[209:239]
d5 = lines[264:267]
l6 = lines[292:321]
d6 = lines[372:375]
l7 = lines[425:453]
d7 = lines[478:481]
l8 = lines[506:530]
d8 = lines[555:558]
l9 = lines[583:614]
d9 = lines[657:662]
d10 = lines[705:711]
tail = lines[753:]

with open('dashboard_CVD.js', 'w+') as file:
    # Escrever gráfico 1 - Publicação x Ano
    for line in l1:
        file.write(line)
    for line in datas['index']:
        file.write('{},'.format(line))
    file.write('\n')
    for line in d1:
        file.write(line)
    for line in datas['ano']:
        file.write('{},'.format(line))
    file.write('\n')
    # Escrever gráfico 2 - Relevância das publicações (COMPLETAR)
    for line in l2:
        file.write(line)
    for line in ['"Total"', '"Com Impacto(FI)"', '"Sem Impacto(FI)"', '"Com Qualis"', '"Sem Qualis"']:
        file.write('{},'.format(line))
    file.write('\n')
    for line in d2:
        file.write(line)
    for line in [len(df), '', '', qualis['qualis'][:8].sum(),len(df)-qualis['qualis'][:8].sum()]:
        file.write('{},'.format(line))
    file.write('\n')
    # Escrever gráfico 3 - Publicação x Qualis
    for line in l3:
        file.write(line)
    for line in qualis['index']:
        file.write('"{}",'.format(line))
    file.write('\n')
    for line in d3:
        file.write(line)
    file.write('\n')
    for line in qualis['qualis']:
        file.write('{},'.format(line))
    file.write('\n')
    # Escrever gráfico 4 - Publicação x FI (COMPLETAR)
    for line in l4:
        file.write(line)
    for line in qualis['index']:
        file.write('"{}",'.format(line))
    file.write('\n')
    for line in d4:
        file.write(line)
    for line in qualis['qualis']:
        file.write('{},'.format(line))
    file.write('\n')
    # Escrever gráfico 5 - Publicação x Citações (COMPLETAR)
    for line in l5:
        file.write(line)
    for line in datas['index']:
        file.write('{},'.format(line))
    file.write('\n')
    for line in d5:
        file.write(line)
    for line in datas['ano']:
        file.write('{},'.format(line))
    file.write('\n')
    # Escrever gráfico 6 - Publicação x H-Index
    for line in l6:
        file.write(line)
    for line in labels_h:
        file.write('{},'.format(line))
    file.write('\n')
    for line in d6:
        file.write(line)
    for line in data_h:
        file.write('{},'.format(line))
    file.write('\n')
    # Escrever gráfico 7 - Farmacos mais citados
    for line in l7:
        file.write(line)
    for line in medicamentos['medicamento']:
        file.write('"{}",'.format(line))
    file.write('\n')
    for line in d7:
        file.write(line)
    for line in medicamentos['contagem']:
        file.write('{},'.format(line))
    file.write('\n')
    # Escrever gráfico 8 - Farmacos mais citados em relevantes
    for line in l8:
        file.write(line)
    for line in medicamentos['medicamento']:
        file.write('"{}",'.format(line))
    file.write('\n')
    for line in d8:
        file.write(line)
    for line in medicamentos['comqualis']:
        file.write('{},'.format(line))
    file.write('\n')
    # Escrever gráfico 9 - Citações prós e contras
    for line in l9:
        file.write(line)
    for line in medicamentos['medicamento']:
        file.write('"{}",'.format(line))
    file.write('\n')
    for line in d9:
        file.write(line)
    for line in medicamentos['positivo']:
        file.write('{},'.format(line))
    file.write('\n')
    for line in d10:
        file.write(line)
    for line in medicamentos['negativo']:
        file.write('{},'.format(line))
    file.write('\n')
    for line in tail:
        file.write(line)

NameError: name 'df' is not defined